In [1]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

In [2]:
data=open("weights.csv").read()

In [3]:
measurements = [x for x in data.split("\n") if x != '']
measurements

dates = [m.split(', ')[0] for m in measurements]
weights = np.array([float(m.split(', ')[1]) for m in measurements])

from datetime import datetime
for k in range(len(dates)):
  dates[k] = datetime.strptime(dates[k], "%Y %m %d %H:%M")

timestamps = []
for k in range(len(dates)):
    timestamps.append(dates[k].timestamp())

In [4]:
from scipy.optimize import curve_fit
from bokeh.plotting import figure, output_notebook, show, save
from bokeh.io import export_png 
output_notebook()

opts = dict(plot_width=900, plot_height=400, min_border=1, x_axis_type='datetime')
p = figure(**opts)

from bokeh.models import DatetimeTickFormatter
p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
)
p.xaxis.major_label_orientation = np.pi/4
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Weight [kg]'

r = p.line(
  dates,
  weights,
  color='red'
)
c = p.circle(dates, weights, fill_color="red", size=8)

def linear(x, a, b):
    return a*x + b

popt, pcov = curve_fit(linear, timestamps, weights)
print(popt)
print(pcov)

tdiff = (timestamps[-1]-timestamps[0])//2
xx = np.linspace(min(timestamps), max(timestamps) + tdiff, 50)
r = p.line(
  [datetime.fromtimestamp(xx[k]) for k in range(len(xx))],
  [linear(xx[k], *popt) for k in range(len(xx))],
  color='blue'
)

print("Average: " + str(np.average(weights)))
print("STD: " + str(np.std(weights)))
print("Min: " + str(np.min(weights)))
print("Max: " + str(np.max(weights)))

show(p)
save(p, filename="plot.html")
export_png(p, filename="plot.png", width=900, height=400)

Loading BokehJS ...

[ 3.15057478e-07 -4.28560766e+02]
[[ 2.24634225e-16 -3.54552710e-07]
 [-3.54552710e-07  5.59630310e+02]]
Average: 68.712
STD: 3.131558078656693
Min: 64.0
Max: 73.5


/Users/nagydaniel/anaconda3/lib/python3.7/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/Users/nagydaniel/anaconda3/lib/python3.7/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/Users/nagydaniel/Documents/myweight/plot.png'